In [ ]:
'''
 * Copyright (c) 2004 Radhamadhab Dalai
 *
 * Permission is hereby granted, free of charge, to any person obtaining a copy
 * of this software and associated documentation files (the "Software"), to deal
 * in the Software without restriction, including without limitation the rights
 * to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
 * copies of the Software, and to permit persons to whom the Software is
 * furnished to do so, subject to the following conditions:
 *
 * The above copyright notice and this permission notice shall be included in
 * all copies or substantial portions of the Software.
 *
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
 * IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
 * FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
 * AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
 * LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
 * OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
 * THE SOFTWARE.
'''

##  Object-Oriented Design for Implementation

In our introduction to linear regression, we walked through various components including the data, the model, the loss function, and the optimization algorithm. Indeed, linear regression is one of the simplest machine learning models. Training it, however, uses many of the same components as other models in this book require. 

### Utilities

We need a few utilities to simplify object-oriented programming in Jupyter notebooks. One of the challenges is that class definitions tend to be fairly long blocks of code. Notebook readability demands short code fragments, interspersed with explanations, a requirement incompatible with the style of programming common for Python libraries.

### Function Registration

The first utility function allows us to register functions as methods in a class after the class has been created. In fact, we can do so even after we have created instances of the class!

```python
import time
import numpy as np
import torch
from torch import nn
from d import torch as d

def add_to_class(Class):
    """Register functions as methods in created class."""
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)
        return obj
    return wrapper
```

### Example Usage

```python
class A:
    def __init__(self):
        self.b = 1

a = A()

@add_to_class(A)
def do(self):
    print('Class attribute "b" is', self.b)

a.do()
```

### Hyperparameter Storage

The second utility is a class that saves all arguments in a class’s `__init__` method as class attributes. 

```python
class HyperParameters:
    """The base class of hyperparameters."""
    def save_hyperparameters(self, ignore=[]):
        raise NotImplemented
```

Usage:

```python
class B(d.HyperParameters):
    def __init__(self, a, b, c):
        self.save_hyperparameters(ignore=['c'])
        print('self.a =', self.a, 'self.b =', self.b)
        print('There is no self.c =', not hasattr(self, 'c'))

b = B(a=1, b=2, c=3)
```

### ProgressBoard for Visualization

A class for interactive plotting of experiment progress:

```python
class ProgressBoard(d2l.HyperParameters):
    """The board that plots data points in animation."""
    def __init__(self, xlabel=None, ylabel=None, xlim=None, ylim=None, 
                 xscale='linear', yscale='linear', ls=['-', '--', '-.', ':'], 
                 colors=['C0', 'C1', 'C2', 'C3'], fig=None, axes=None, 
                 figsize=(3.5, 2.5), display=True):
        self.save_hyperparameters()
    
    def draw(self, x, y, label, every_n=1):
        raise NotImplemented
```

Example Usage:

```python
board = d.ProgressBoard('x')
for x in np.arange(0, 10, 0.1):
    board.draw(x, np.sin(x), 'sin', every_n=2)
    board.draw(x, np.cos(x), 'cos', every_n=10)
```


##  Object-Oriented Design for Implementation

In our introduction to linear regression, we walked through various components including the data, the model, the loss function, and the optimization algorithm. Indeed, linear regression is one of the simplest machine learning models. Training it, however, uses many of the same components as other models in this book require. Therefore, before diving into the implementation details, it is worth designing some of the APIs used throughout this book.

### Object-Oriented Design

Treating components in deep learning as objects, we can start by defining classes for these objects and their interactions. This object-oriented design will streamline implementation and make projects more modular. Inspired by open-source libraries such as PyTorch Lightning, we define three classes:

1. **Module**: Contains models, losses, and optimization methods.
2. **DataModule**: Provides data loaders for training and validation.
3. **Trainer**: Combines the above two classes and facilitates training on various hardware platforms.

Most code will adapt **Module** and **DataModule**, while **Trainer** will be discussed when we cover GPUs, CPUs, and parallel training.

```python
import time
import numpy as np
import torch
from torch import nn
from d import torch as d
```

##  Utilities

To simplify object-oriented programming in Jupyter notebooks, we introduce a few utilities.

### Registering Methods Dynamically

The first utility function allows us to register functions as methods in a class after it has been created. This enables splitting class implementation into multiple code blocks.

```python
def add_to_class(Class):
    """Register functions as methods in created class."""
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)
        return obj
    return wrapper
```

#### Example Usage

```python
class A:
    def __init__(self):
        self.b = 1

a = A()

@add_to_class(A)
def do(self):
    print('Class attribute "b" is', self.b)

a.do()
```

### Hyperparameter Management

The next utility class saves all arguments in a class's `__init__` method as class attributes.

```python
class HyperParameters:
    """Base class for managing hyperparameters."""
    def save_hyperparameters(self, ignore=[]):
        raise NotImplemented
```

#### Example Usage

```python
class B(d.HyperParameters):
    def __init__(self, a, b, c):
        self.save_hyperparameters(ignore=['c'])
        print('self.a =', self.a, 'self.b =', self.b)
        print('There is no self.c =', not hasattr(self, 'c'))

b = B(a=1, b=2, c=3)
```

### Progress Visualization

We also introduce a utility class `ProgressBoard` that allows plotting experiment progress in real-time.

```python
class ProgressBoard(d.HyperParameters):
    """Plots data points interactively during training."""
    def __init__(self, xlabel=None, ylabel=None, xlim=None, ylim=None, xscale='linear', yscale='linear',
                 ls=['-', '--', '-.', ':'], colors=['C0', 'C1', 'C2', 'C3'],
                 fig=None, axes=None, figsize=(3.5, 2.5), display=True):
        self.save_hyperparameters()

    def draw(self, x, y, label, every_n=1):
        raise NotImplemented
```

#### Example Usage
```python

board = d.ProgressBoard('x')
for x in np.arange(0, 10, 0.1):
    board.draw(x, np.sin(x), 'sin', every_n=2)
    board.draw(x, np.cos(x), 'cos', every_n=10)
```

This utility will enable real-time visualization of training progress, providing insights into model convergence and performance.

---

This structured approach ensures modularity, readability, and ease of experimentation in deep learning projects.

